# Performing Read Alignment

Here we will use the BWA aligner to align a small set of Illumina sequencing data to the _human_ reference genome. 

First, check you are in the correct directory.

In [ ]:
pwd

It should display something like:

  ``NGScourse_22/alignment``

## Viewing the reference genome

view the `ref` directory that contains the fasta files of the reference genomes:

In [ ]:
ls /home/ref

Fasta files (.fa) are used to store raw sequencing information before aligning data. Human whole genome references is contained in the file Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

View the file with zless (we use zless instead of less because the file is compressed):

In [ ]:
zless /home/ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa

Similar to a BAM file, to allow fast retrieval of data, an index file is often required. You should check for the presencen of fasta indexes for the genome in the 'ref' directory:

``Homo_sapiens.GRCh38.dna.primary_assembly.fa.amb ... Homo_sapiens.GRCh38.dna.primary_assembly.fa.sa``

These are created by BWA: suffixtrees, bwt transform etc etc.
    
If these index files don’t exist, then you can run the indexing with the command

``bwa index Homo_sapiens.GRCh38.dna.primary_assembly.fa`` 
    
Beware – this indexing process can take 3-5 minutes so please only run it if the index files do not exist!

## Align the data with bwa

Go to the `data/` directory - you can use this command: 

In [ ]:
cd data/

Use the `bwa mem` command to align the fastq files to the human reference genome. By default bwa outputs SAM format directly to the standard output (in this case your terminal window), therefore you will have to redirect the result into a SAM file.

In [ ]:
bwa mem /home/ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa SRR13882963_1P.fastq SRR13882963_2P.fastq > SRR13882963.sam

This may take a few minutes, please be patient.

## Convert a SAM file to a BAM file

Now use samtools to convert the SAM file `SRR13882963.sam` created in the previous step into a BAM file called `SRR13882963.bam`.

In [ ]:
samtools view -O BAM -o SRR13882963.bam SRR13882963.sam

__Q1: How much space is saved by using a BAM file instead of a SAM file?__

## Sort and index the BAM file

The BAM files produced by BWA are sorted by read name (same order as the original fastq files). However, most viewing and variant calling software require the BAM files to be sorted by reference coordinate position and indexed for rapid retrieval. Therefore, use ‘samtools sort’ to produce a new BAM file called `SRR13882963.sorted.bam` that is sorted by position.

In [ ]:
samtools sort -T temp -O bam -o SRR13882963.sorted.bam SRR13882963.bam

Finally index the sorted BAM file using ‘samtools index’ command. 

__Note:__ indexing a BAM file is also a good way to check that the BAM file has not been truncated (e.g. your disk becomes full when writing the BAM file). At the end of every BAM file, a special end of file (EOF) marker is written. The Samtools index command will first check for this and produce an error message if it is not found.

In [ ]:
samtools index SRR13882963.sorted.bam

## Unix pipes to combine the commands together

To produce the sorted BAM file above we had to carry out several separate commands and produce intermediate files. The Unix pipe command allows you to feed the output of one command into the next command.

You can combine all of these commands together using unix pipes, and do all of this data processing together and avoid writing intermediate files. To do this type:

In [ ]:
bwa mem /home/ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa SRR13882963_1P.fastq SRR13882963_2P.fastq | samtools view -O BAM - | samtools sort -T temp -O bam -o SRR13882963_2.sorted.bam -

Now index the BAM file:

In [ ]:
samtools index SRR13882963_2.sorted.bam

**Note: When the symbol `-` is used above, Unix will automatically replace `-` with the output produced by the preceeding command (i.e. the command before the `|` symbol).**